<!--NOTEBOOK_HEADER-->
*This notebook contains material from [cbe-virtual-laboratory](https://jckantor.github.io/cbe-virtual-laboratory);
content is available [on Github](https://github.com/jckantor/cbe-virtual-laboratory.git).*


<!--NAVIGATION-->
< [2.0 Proof of Concept](https://jckantor.github.io/cbe-virtual-laboratory/02.00-Proof_of_Concept.html) | [Contents](toc.html) | [2.2 Remote Operation of a Laboratory Experiment via MQTT](https://jckantor.github.io/cbe-virtual-laboratory/02.02-Remote_Operation_of_a_Laboratory_Experiment.html) ><p><a href="https://colab.research.google.com/github/jckantor/cbe-virtual-laboratory/blob/master/docs/02.01-SImulation_of_a_Laboratory_Experiment_Published_via_MQTT.ipynb"> <img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open in Google Colaboratory"></a><p><a href="https://jckantor.github.io/cbe-virtual-laboratory/02.01-SImulation_of_a_Laboratory_Experiment_Published_via_MQTT.ipynb"> <img align="left" src="https://img.shields.io/badge/Github-Download-blue.svg" alt="Download" title="Download Notebook"></a>

# 2.1 Simulation of a Remote Laboratory Experiment Published via MQTT

This notebook is part of a proof concept study regarding use of the Internet of Things (IoT) as infrastructure for engineering teaching laboratories. 

This notebook uses the [SimPy](https://simpy.readthedocs.io/en/latest/) library a create real-time simulation of a hypothetical experiment. The experiment responds to inputs and publishes data to a remote user via an an MQTT broker. In actual use, the content of this notebook would be implemented in an attached device such as an Arduino, Particle, or Raspberry Pi, with attached sensors.

The companion notebook ??? demonstrates how a remote client could interact with the experiment via MQTT.

## 2.1.1 Installations

The following installations are required for use on Google Colab.

In [2]:
!pip install paho-mqtt
!pip install simpy

     |████████████████████████████████| 102kB 5.0MB/s ta 0:00:011
  Created wheel for paho-mqtt: filename=paho_mqtt-1.5.1-cp36-none-any.whl size=61544 sha256=e1c78ce2e63e037c45f8aff66b8e2a46e045b79449c8b7f1f20746a5f394f4cd
  Stored in directory: /root/.cache/pip/wheels/75/e2/f5/78942b19b4d135605e58dfe85fba52253b14d636aabf76904b
Successfully built paho-mqtt


## 2.1.2 Publishing a real-time simulation via MQTT

Topics:

| topic | messages |
| :-- | :-- |
| `cbe-virtual-lab/command` | start and stop experiments |
| `cbe-virtual-lab/expt-name/data` | topic


### 2.1.2.1 Proof of Concept

Here we experiment with encapsulating the experiment as a standalone class. This is set up so that upon receiving an appropriate message from the remote user, a new instance of the experiment is created and run.

In [2]:
import time
import simpy
import paho.mqtt.publish as publish
import paho.mqtt.client as mqtt
import json

class Experiment():

    def __init__(self, topic, duration):
        self.topic = topic
        self.duration = duration
        self.env = simpy.rt.RealtimeEnvironment(factor=1)
        self.proc = self.env.process(self.process())
        self.client = mqtt.Client()
        #self.client.on_connect = self.on_connect
        self.client.on_publish = self.on_publish

    def on_connect(self, client, userdata, flags, rc):
        print(f"Connected with result code {rc}")
        print(client.clientId)
        self.client.subscribe(self.topic)

    def on_publish(self, client, userdata, result):
        print(f"{client} published with result code {result}")

    def process(self):
        t_start = time.perf_counter()
        t = 0
        y = 2.0
        while True:
            msg = f"{round(t,2)},{y:5.2f}"
            self.client.publish(self.topic, msg)
            yield self.env.timeout(1 - (t - round(t, 0)))
            t = time.perf_counter() - t_start
            y -= 0.1*y

    def run(self, client):
        print(f"Experiment started by {client}")
        self.client.connect("mqtt.eclipse.org", 1883, 60)
        self.env.run(until=self.duration)
        self.client.disconnect()
        print("End experiment.")

# set up client to wait for command messages

def on_connect(client, userdata, flags, rc):
    """report connection and subscribe"""
    print(f"Connected with result code {rc}")
    client.subscribe("cbe-virtual-lab/command/#")

def on_message(client, userdata, msg):
    """setup and start new experiment"""
    print(f"Received {msg.payload} from {msg.topic}")
    data = json.loads(msg.payload.decode("utf-8"))
    duration = data['duration']
    topic = data['topic']
    expt = Experiment(topic, duration)
    expt.run(client)

# setup client
client = mqtt.Client("virtual-cbe-lab")
client.on_connect = on_connect
client.on_message = on_message

# connect client to broker
client.connect("mqtt.eclipse.org", 1883, 60)

# start a non-blocking thread to wait for messages
client.loop_start()

# prove the loop is non-blocking
for k in range(20):
    print(k)
    time.sleep(3)

# don't leave a zombie thread behind
client.loop_stop()

0
Connected with result code 0
1
Received b'{"duration": 10, "topic": "cbe-virtual-lab/tank-level/simulation"}' from cbe-virtual-lab/command
Experiment started by <paho.mqtt.client.Client object at 0x7f3cfb071a20>
<paho.mqtt.client.Client object at 0x7f3cfb071dd8> published with result code 1
<paho.mqtt.client.Client object at 0x7f3cfb071dd8> published with result code 2
2
<paho.mqtt.client.Client object at 0x7f3cfb071dd8> published with result code 3
<paho.mqtt.client.Client object at 0x7f3cfb071dd8> published with result code 4
<paho.mqtt.client.Client object at 0x7f3cfb071dd8> published with result code 5
3
<paho.mqtt.client.Client object at 0x7f3cfb071dd8> published with result code 6
<paho.mqtt.client.Client object at 0x7f3cfb071dd8> published with result code 7
<paho.mqtt.client.Client object at 0x7f3cfb071dd8> published with result code 8
4
<paho.mqtt.client.Client object at 0x7f3cfb071dd8> published with result code 9
<paho.mqtt.client.Client object at 0x7f3cfb071dd8> published

<!--NAVIGATION-->
< [2.0 Proof of Concept](https://jckantor.github.io/cbe-virtual-laboratory/02.00-Proof_of_Concept.html) | [Contents](toc.html) | [2.2 Remote Operation of a Laboratory Experiment via MQTT](https://jckantor.github.io/cbe-virtual-laboratory/02.02-Remote_Operation_of_a_Laboratory_Experiment.html) ><p><a href="https://colab.research.google.com/github/jckantor/cbe-virtual-laboratory/blob/master/docs/02.01-SImulation_of_a_Laboratory_Experiment_Published_via_MQTT.ipynb"> <img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open in Google Colaboratory"></a><p><a href="https://jckantor.github.io/cbe-virtual-laboratory/02.01-SImulation_of_a_Laboratory_Experiment_Published_via_MQTT.ipynb"> <img align="left" src="https://img.shields.io/badge/Github-Download-blue.svg" alt="Download" title="Download Notebook"></a>